In [ ]:
import qiskit

In [ ]:
from qiskit import QuantumCircuit, Aer, execute
from qiskit.visualization import plot_histogram
import numpy as np

# Set random seed for reproducibility
np.random.seed(42)

# Parameters
n_bits = 4  # Number of bits for the key
eavesdrop = True  # Toggle Eve's presence (True/False)

# Step 1: Alice generates random bits and bases
alice_bits = np.random.randint(2, size=n_bits)  # Random 0s and 1s
alice_bases = np.random.randint(2, size=n_bits)  # 0 for rectilinear (+), 1 for diagonal (×)

# Step 2: Alice prepares qubits
circuits = []
for i in range(n_bits):
    qc = QuantumCircuit(1, 1)  # 1 qubit, 1 classical bit
    
    # Encode bit: 0 -> |0⟩, 1 -> |1⟩
    if alice_bits[i] == 1:
        qc.x(0)  # Apply X gate to flip to |1⟩
    
    # Choose basis: 0 -> rectilinear (|0⟩, |1⟩), 1 -> diagonal (|+⟩, |-⟩)
    if alice_bases[i] == 1:
        qc.h(0)  # Apply Hadamard gate for diagonal basis
    
    circuits.append(qc)

# Step 3: Eve eavesdrops (optional)
if eavesdrop:
    eve_bases = np.random.randint(2, size=n_bits)  # Eve's random bases
    for i in range(n_bits):
        if eve_bases[i] == 1:
            circuits[i].h(0)  # Eve measures in diagonal basis
        circuits[i].measure(0, 0)  # Eve measures
        # Simulate collapse (re-prepare state based on Eve's measurement)
        job = execute(circuits[i], Aer.get_backend('qasm_simulator'), shots=1)
        result = job.result().get_counts()
        measured_bit = int(list(result.keys())[0])  # Eve's measured bit
        circuits[i] = QuantumCircuit(1, 1)  # Reset circuit
        if measured_bit == 1:
            circuits[i].x(0)
        if eve_bases[i] == 1:
            circuits[i].h(0)

# Step 4: Bob measures in random bases
bob_bases = np.random.randint(2, size=n_bits)  # Bob's random bases
bob_results = []
for i in range(n_bits):
    # Bob measures in his basis
    if bob_bases[i] == 1:
        circuits[i].h(0)  # Diagonal basis
    circuits[i].measure(0, 0)
    
    # Simulate measurement
    job = execute(circuits[i], Aer.get_backend('qasm_simulator'), shots=1)
    result = job.result().get_counts()
    bob_results.append(int(list(result.keys())[0]))

# Step 5: Alice and Bob compare bases and extract key
final_key_alice = []
final_key_bob = []
for i in range(n_bits):
    if alice_bases[i] == bob_bases[i]:  # Matching bases
        final_key_alice.append(alice_bits[i])
        final_key_bob.append(bob_results[i])

# Step 6: Output results
print("Alice's bits:    ", alice_bits)
print("Alice's bases:  ", alice_bases)
print("Bob's bases:    ", bob_bases)
print("Bob's results:  ", bob_results)
print("Final key (Alice):", final_key_alice)
print("Final key (Bob):  ", final_key_bob)

# Check for errors (due to eavesdropping)
if final_key_alice == final_key_bob:
    print("Keys match! No eavesdropping detected.")
else:
    print("Keys differ! Eavesdropping suspected.")

# Optional: Visualize one circuit (e.g., the first one)
print("\nExample circuit (first qubit):")
print(circuits[0])